In [5]:
!sudo mkdir files3
!gsutil -m rsync -r gs://cnn_chips files3

/home


In [6]:
cd /home

/home


In [16]:
ls files3/S1

Bolivia_103757_S1.tif  Mekong_221789_S1.tif     Somalia_7931_S1.tif
Bolivia_129334_S1.tif  Mekong_254910_S1.tif     Somalia_886726_S1.tif
Bolivia_195474_S1.tif  Mekong_269835_S1.tif     Somalia_93023_S1.tif
Bolivia_23014_S1.tif   Mekong_293769_S1.tif     Somalia_94102_S1.tif
Bolivia_233925_S1.tif  Mekong_333434_S1.tif     Somalia_970508_S1.tif
Bolivia_242570_S1.tif  Mekong_342411_S1.tif     Somalia_989553_S1.tif
Bolivia_290290_S1.tif  Mekong_382276_S1.tif     Somalia_992457_S1.tif
Bolivia_294583_S1.tif  Mekong_424793_S1.tif     Spain_1167260_S1.tif
Bolivia_312675_S1.tif  Mekong_45934_S1.tif      Spain_1199913_S1.tif
Bolivia_314919_S1.tif  Mekong_474783_S1.tif     Spain_2472849_S1.tif
Bolivia_360519_S1.tif  Mekong_52610_S1.tif      Spain_2523247_S1.tif
Bolivia_379434_S1.tif  Mekong_596495_S1.tif     Spain_2594119_S1.tif
Bolivia_432776_S1.tif  Mekong_774566_S1.tif     Spain_2938657_S1.tif
Bolivia_60373_S1.tif   Mekong_846007_S1.tif     Spain_3285448_S1.tif
Bolivia_76104_S1.tif   Mekong_9

In [7]:
!pip install rasterio

In [20]:
from time import time
import csv
from PIL import Image
import rasterio
import numpy as np
import os
BASEDIR = 'files3/'

def getArrFlood(fname):
  return rasterio.open(fname).read()

def download_flood_water_data_from_list(l):
  i= 0
  tot_nan = 0
  tot_good = 0
  flood_data = []
  for (im_fname, mask_fname) in l:
    if not os.path.exists(os.path.join("files3/", im_fname)):
      print("skipping", os.path.join("files3/", im_fname))
      continue
    arr_x = np.nan_to_num(getArrFlood(os.path.join("files3/", im_fname)))
    arr_y = np.uint8(getArrFlood(os.path.join("files3/", mask_fname)))
    ignore = (arr_y == -1)
    ignore = np.uint8(((np.uint8(ignore) * -1) * 256) + 1)
    arr_y *= ignore
    arr_y = np.uint8(getArrFlood(os.path.join("files3/", mask_fname)))
    if np.sum((arr_y != arr_y)) == 0:
      arr_x = np.clip(arr_x, -50, 1)
      arr_x = (arr_x + 50) / 51
      if i % 100 == 0:
        print(i)
        print(im_fname, mask_fname)
      i += 1
      flood_data.append((arr_x,arr_y))
    else:
      print("skipping nan")
  print(i)
  return flood_data

def download_flood_water_classification_data_from_list(l):
  i= 0
  tot_nan = 0
  tot_good = 0
  flood_data = []
  for (im_fname, mask_fname) in l:
    if not os.path.exists(os.path.join("files3/", im_fname)):
      print(os.path.join("files3/", im_fname))
      continue
    arr_x = np.nan_to_num(getArrFlood(os.path.join("files3/", im_fname)))
    arr_y = np.uint8(getArrFlood(os.path.join("files3/", mask_fname)))
    ignore = (arr_y == -1)
    ignore = np.uint8(((np.uint8(ignore) * -1) * 256) + 1)
    arr_y *= ignore
    arr_y = np.uint8(getArrFlood(os.path.join("files3/", mask_fname)))
    if np.sum((arr_y != arr_y)) == 0:
      if i % 100 == 0:
        print(i)
        print(im_fname, mask_fname)
      i += 1
      flood_data.append((arr_x,arr_y))
    else:
      print("skipping nan")
  print(i)
  return flood_data


def load_flood_train_data():
  fname = "files3/flood_train_data.csv"
  with open(fname) as f:
    fname = [tuple(line) for line in csv.reader(f)]
  return download_flood_water_data_from_list(fname)

def load_weak_flood_train_data():
  files = [(os.path.join("S1_NoQC", x[1]), os.path.join("NoQC", x[0])) for x in zip(sorted(os.listdir("files4/NoQC")), sorted(os.listdir("files4/S1_NoQC")))]
  files = [x for x in files if "Bolivia" not in x[0]]
  print(files[0:10])
  return download_flood_water_data_from_list(files)

def load_weak_flood_S1_train_data():
  files = [(os.path.join("S1_NoQC", x[1]), os.path.join("S1Flood_NoQC", x[0])) for x in zip(sorted(os.listdir("files4/S1Flood_NoQC")), sorted(os.listdir("files4/S1_NoQC")))]
  print(files[0:10])
  return download_flood_water_data_from_list(files)


def load_flood_valid_data():
  fname = "files3/flood_valid_data.csv"
  with open(fname) as f:
    fname = [tuple(line) for line in csv.reader(f)]
  print(fname, "files!")
  return download_flood_water_data_from_list(fname)

def load_flood_test_data():
  fname = "files3/flood_test_data.csv"
  with open(fname) as f:
    fname = [tuple(line) for line in csv.reader(f)]
  return download_flood_water_data_from_list(fname)

def load_flood_test_perm_data():
  fname = "files3/flood_test_data.csv"
  with open(fname) as f:
    fname = [tuple(line) for line in csv.reader(f)]
  fname = [(t[0], t[1].replace("QC_v2", "Perm").replace("QC", "Perm")) for t in fname]
  return download_flood_water_data_from_list(fname)

def load_flood_bolivia_test_data():
  fname = "files3/flood_bolivia_data.csv"
  with open(fname) as f:
    fname = [tuple(line) for line in csv.reader(f)]
  return download_flood_water_data_from_list(fname)

def load_flood_bolivia_test_perm_data():
  fname = "files3/flood_bolivia_data.csv"
  with open(fname) as f:
    fname = [tuple(line) for line in csv.reader(f)]
  fname = [(t[0], t[1].replace("QC_v2", "Perm").replace("QC", "Perm")) for t in fname]
  return download_flood_water_data_from_list(fname)

def load_flood_test_trad_data():
  fname = "files3/flood_test_data.csv"
  with open(fname) as f:
    fname = [tuple(line) for line in csv.reader(f)]
  fname = [getTradFname(x) for x in fname] 
  return download_flood_water_classification_data_from_list(fname)

def load_flood_bolivia_test_trad_data():
  fname = "files3/flood_bolivia_data.csv"
  with open(fname) as f:
    fname = [tuple(line) for line in csv.reader(f)]
  fname = [getTradFname(x) for x in fname] 
  return download_flood_water_classification_data_from_list(fname)

def getTradFname(a):
  x,y = a
  x = x.replace("S1", "S1Flood")
  return x,y


In [21]:
flood_test_perm_data = load_flood_test_perm_data()
flood_test_all_data = load_flood_test_data()

flood_test_bolivia_perm_data = load_flood_bolivia_test_perm_data()
flood_test_bolivia_all_data = load_flood_bolivia_test_data()

0
S1/Ghana_313799_S1.tif Perm/Ghana_313799_Perm.tif
90
0
S1/Ghana_313799_S1.tif QC_v2/Ghana_313799_QC.tif
90
0
S1/Bolivia_103757_S1.tif Perm/Bolivia_103757_Perm.tif
15
0
S1/Bolivia_103757_S1.tif QC_v2/Bolivia_103757_QC.tif
15


In [25]:
def ignore_perm_water(all_water, perm_water):
  w = all_water.copy()
  perm_water = (w == 1) * (perm_water == 1)
  w[perm_water] = 255
  return w

def ignore_flood_water(all_water, perm_water):
  w = all_water.copy()
  flood_water = (w == 1) * (perm_water == 0)
  w[flood_water] = 255
  return w

def get_flood_flood_test_data(all_water_data, perm_water_data):
  flood_flood_test_data = []
  for ((awx, awy), (pwx, pwy)) in zip(all_water_data, perm_water_data):
    pwy = ignore_perm_water(awy, pwy)
    if np.sum(awy == 1) > 0 and np.sum(pwy == 1) == 0:
      continue
    flood_flood_test_data.append((awx, pwy))
  return flood_flood_test_data
  

def get_perm_flood_test_data(all_water_data, perm_water_data):
  perm_flood_test_data = []
  for ((awx, awy), (pwx, pwy)) in zip(all_water_data, perm_water_data):
    pwy = ignore_flood_water(awy, pwy)
    if np.sum(awy == 1) > 0 and np.sum(pwy == 1) == 0:
      continue
    perm_flood_test_data.append((awx, pwy))
  return perm_flood_test_data


In [26]:
flood_flood_test_data = get_flood_flood_test_data(flood_test_all_data, flood_test_perm_data)
perm_flood_test_data = get_perm_flood_test_data(flood_test_all_data, flood_test_perm_data)

flood_flood_bolivia_test_data = get_flood_flood_test_data(flood_test_bolivia_all_data, flood_test_bolivia_perm_data)
perm_flood_bolivia_test_data = get_perm_flood_test_data(flood_test_bolivia_all_data, flood_test_bolivia_perm_data)

In [27]:
import torch
from torchvision import transforms
import torchvision.transforms.functional as F
import random

class InMemoryDataset(torch.utils.data.Dataset):
  
  def __init__(self, data_list, preprocess_func):
    self.data_list = data_list
    self.preprocess_func = preprocess_func
  
  def __getitem__(self, i):
    return self.preprocess_func(self.data_list[i])
  
  def __len__(self):
    return len(self.data_list)


def processAndAugment(data):
  (x,y) = data
  im,label = x.copy(), y.copy()

  # convert to PIL for easier transforms
  im1 = Image.fromarray(im[0])
  im2 = Image.fromarray(im[1])
  label = Image.fromarray(label.squeeze())

  # Get params for random transforms
  i, j, h, w = transforms.RandomCrop.get_params(im1, (256, 256))
  
  im1 = F.crop(im1, i, j, h, w)
  im2 = F.crop(im2, i, j, h, w)
  label = F.crop(label, i, j, h, w)
  if random.random() > 0.5:
    im1 = F.hflip(im1)
    im2 = F.hflip(im2)
    label = F.hflip(label)
  if random.random() > 0.5:
    im1 = F.vflip(im1)
    im2 = F.vflip(im2)
    label = F.vflip(label)
  
  norm = transforms.Normalize([0.6851, 0.5235], [0.0820, 0.1102])
  im = torch.stack([transforms.ToTensor()(im1).squeeze(), transforms.ToTensor()(im2).squeeze()])
  im = norm(im)
  label = transforms.ToTensor()(label).squeeze()
  if torch.sum(label.gt(.003) * label.lt(.004)):
    label *= 255
  label = label.round()

  return im, label


def processTestIm(data):
  (x,y) = data
  im,label = x.copy(), y.copy()
  norm = transforms.Normalize([0.6851, 0.5235], [0.0820, 0.1102])
  #label[0][0][0] = 255
  # convert to PIL for easier transforms
  im_c1 = Image.fromarray(im[0]).resize((512,512))
  im_c2 = Image.fromarray(im[1]).resize((512,512))
  label = Image.fromarray(label.squeeze()).resize((512,512))

  im_c1s = [F.crop(im_c1, 0, 0, 256, 256), F.crop(im_c1, 0, 256, 256, 256),
            F.crop(im_c1, 256, 0, 256, 256), F.crop(im_c1, 256, 256, 256, 256)]
  im_c2s = [F.crop(im_c2, 0, 0, 256, 256), F.crop(im_c2, 0, 256, 256, 256),
            F.crop(im_c2, 256, 0, 256, 256), F.crop(im_c2, 256, 256, 256, 256)]
  labels = [F.crop(label, 0, 0, 256, 256), F.crop(label, 0, 256, 256, 256),
            F.crop(label, 256, 0, 256, 256), F.crop(label, 256, 256, 256, 256)]


  ims = [torch.stack((transforms.ToTensor()(x).squeeze(),
                    transforms.ToTensor()(y).squeeze()))
                    for (x,y) in zip(im_c1s, im_c2s)]
  ims = [norm(im) for im in ims]
  ims = torch.stack(ims)
  labels = [(transforms.ToTensor()(label).squeeze()) for label in labels]
  labels = torch.stack(labels)
  if torch.sum(labels.gt(.003) * labels.lt(.004)):
    labels *= 255
  labels = labels.round()
  return ims, labels


In [28]:
test_all_data = flood_test_all_data
test_all_dataset = InMemoryDataset(test_all_data, processTestIm)
test_all_loader = torch.utils.data.DataLoader(test_all_dataset, batch_size=1, shuffle=True, sampler=None,
                  batch_sampler=None, num_workers=0, collate_fn=lambda x: x[0],
                  pin_memory=True, drop_last=False, timeout=0,
                  worker_init_fn=None)
test_all_iter = iter(test_all_loader)

test_flood_data = flood_flood_test_data
test_flood_dataset = InMemoryDataset(test_flood_data, processTestIm)
test_flood_loader = torch.utils.data.DataLoader(test_flood_dataset, batch_size=1, shuffle=True, sampler=None,
                  batch_sampler=None, num_workers=0, collate_fn=lambda x: x[0],
                  pin_memory=True, drop_last=False, timeout=0,
                  worker_init_fn=None)
test_flood_iter = iter(test_flood_loader)

test_perm_data = perm_flood_test_data
test_perm_dataset = InMemoryDataset(test_perm_data, processTestIm)
test_perm_loader = torch.utils.data.DataLoader(test_perm_dataset, batch_size=1, shuffle=True, sampler=None,
                  batch_sampler=None, num_workers=0, collate_fn=lambda x: x[0],
                  pin_memory=True, drop_last=False, timeout=0,
                  worker_init_fn=None)
test_perm_iter = iter(test_perm_loader)

In [29]:
test_bolivia_all_data = flood_test_bolivia_all_data
test_bolivia_all_dataset = InMemoryDataset(test_bolivia_all_data, processTestIm)
test_bolivia_all_loader = torch.utils.data.DataLoader(test_bolivia_all_dataset, batch_size=1, shuffle=True, sampler=None,
                  batch_sampler=None, num_workers=0, collate_fn=lambda x: x[0],
                  pin_memory=True, drop_last=False, timeout=0,
                  worker_init_fn=None)
test_bolivia_all_iter = iter(test_bolivia_all_loader)

test_bolivia_flood_data = flood_flood_bolivia_test_data
test_bolivia_flood_dataset = InMemoryDataset(test_bolivia_flood_data, processTestIm)
test_bolivia_flood_loader = torch.utils.data.DataLoader(test_bolivia_flood_dataset, batch_size=1, shuffle=True, sampler=None,
                  batch_sampler=None, num_workers=0, collate_fn=lambda x: x[0],
                  pin_memory=True, drop_last=False, timeout=0,
                  worker_init_fn=None)
test_bolivia_flood_iter = iter(test_flood_loader)

test_bolivia_perm_data = perm_flood_bolivia_test_data
test_bolivia_perm_dataset = InMemoryDataset(test_bolivia_perm_data, processTestIm)
test_bolivia_perm_loader = torch.utils.data.DataLoader(test_bolivia_perm_dataset, batch_size=1, shuffle=True, sampler=None,
                  batch_sampler=None, num_workers=0, collate_fn=lambda x: x[0],
                  pin_memory=True, drop_last=False, timeout=0,
                  worker_init_fn=None)
test_bolivia_perm_iter = iter(test_bolivia_perm_loader)

In [30]:
import torchvision.models as models
import torch.nn as nn
import torch

net = models.segmentation.fcn_resnet50(pretrained=False, num_classes=2, pretrained_backbone=False)
net.backbone.conv1 = nn.Conv2d(2, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
def convertBNtoGN(module, num_groups=16):
  if isinstance(module, torch.nn.modules.batchnorm.BatchNorm2d):
    return nn.GroupNorm(num_groups, module.num_features,
                        eps=module.eps, affine=module.affine)
    if module.affine:
        mod.weight.data = module.weight.data.clone().detach()
        mod.bias.data = module.bias.data.clone().detach()

  for name, child in module.named_children():
      module.add_module(name, convertBNtoGN(child, num_groups=num_groups))

  return module

net = convertBNtoGN(net)
net

FCN(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(2, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): GroupNorm(16, 64, eps=1e-05, affine=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): GroupNorm(16, 64, eps=1e-05, affine=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): GroupNorm(16, 64, eps=1e-05, affine=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): GroupNorm(16, 256, eps=1e-05, affine=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): GroupNorm(16, 256, eps=1e-05, affine=True)
        )
  

In [32]:
net.load_state_dict(torch.load("files3/cnn_checkpoints/5e4_flood_0_89_0.5418742895126343.cp"))

<All keys matched successfully>

In [33]:
from tqdm import tqdm_notebook as tqdm
from IPython.display import clear_output

def computeIOU(output, target):
  output = torch.argmax(output, dim=1).flatten() 
  target = target.flatten()
  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  intersection = torch.sum(output * target)
  union = torch.sum(target) + torch.sum(output) - intersection
  iou = (intersection + .0000001) / (union + .0000001)
  if iou != iou:
    print("failed, replacing with 0")
    iou = torch.tensor(0).float()
  return iou
  

def computeAccuracy(output, target):
  output = torch.argmax(output, dim=1).flatten() 
  target = target.flatten()
  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  correct = torch.sum(output.eq(target))
  return correct.float() / len(target)
  
def truePositives(output, target):
  output = torch.argmax(output, dim=1).flatten() 
  target = target.flatten()
  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  correct = torch.sum(output * target)
  return correct

def trueNegatives(output, target):
  output = torch.argmax(output, dim=1).flatten() 
  target = target.flatten()
  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  output = (output == 0)
  target = (target == 0)
  correct = torch.sum(output * target)
  return correct

def falsePositives(output, target):
  output = torch.argmax(output, dim=1).flatten() 
  target = target.flatten()
  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  output = (output == 1)
  target = (target == 0)
  correct = torch.sum(output * target)
  return correct

def falseNegatives(output, target):
  output = torch.argmax(output, dim=1).flatten() 
  target = target.flatten()
  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  output = (output == 0)
  target = (target == 1)
  correct = torch.sum(output * target)
  return correct


def test_loop(test_data_loader, net):
  net = net.eval()
  net = net.cuda()
  count = 0
  iou = 0
  loss = 0
  accuracy = 0
  tp = 0
  fp = 0
  tn = 0
  fn = 0
  with torch.no_grad():
      for (images, labels) in tqdm(test_data_loader):
          net = net.cuda()
          outputs = net(images.cuda())
          valid_iou = computeIOU(outputs["out"], labels.cuda())
          iou += valid_iou
          accuracy += computeAccuracy(outputs["out"], labels.cuda())
          tp +=  truePositives(outputs["out"], labels.cuda())
          fp +=  falsePositives(outputs["out"], labels.cuda())
          tn +=  trueNegatives(outputs["out"], labels.cuda())
          fn +=  falseNegatives(outputs["out"], labels.cuda())
          count += 1

  iou = iou / count
  print("Test Mean IOU:", iou)
  print("Total IOU:", (tp.float() / (fn + fp + tp)))
  print("OMISSON:", fn.float() / (fn + tp))
  print("COMMISSON:", fp.float() / (tn + fp))
  print("Test Accuracy:", accuracy / count)

net.load_state_dict(torch.load("files3/cnn_checkpoints/5e4_flood_0_89_0.5418742895126343.cp"))
test_loop(test_all_loader, net)
test_loop(test_flood_loader, net)
test_loop(test_perm_loader, net)
print("\n\nBOLIVIA\n")
test_loop(test_bolivia_all_loader, net)
test_loop(test_bolivia_flood_loader, net)
test_loop(test_bolivia_perm_loader, net)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:83: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Test Mean IOU: tensor(0.4084, device='cuda:0')
Total IOU: tensor(0.6621, device='cuda:0')
OMISSON: tensor(0.1575, device='cuda:0')
COMMISSON: tensor(0.0414, device='cuda:0')
Test Accuracy: tensor(0.9384, device='cuda:0')



Test Mean IOU: tensor(0.3389, device='cuda:0')
Total IOU: tensor(0.5656, device='cuda:0')
OMISSON: tensor(0.2155, device='cuda:0')
COMMISSON: tensor(0.0414, device='cuda:0')
Test Accuracy: tensor(0.9277, device='cuda:0')



Test Mean IOU: tensor(0.3818, device='cuda:0')
Total IOU: tensor(0.6701, device='cuda:0')
OMISSON: tensor(0.0588, device='cuda:0')
COMMISSON: tensor(0.0467, device='cuda:0')
Test Accuracy: tensor(0.9336, device='cuda:0')


BOLIVIA




Test Mean IOU: tensor(0.3160, device='cuda:0')
Total IOU: tensor(0.4750, device='cuda:0')
OMISSON: tensor(0.2482, device='cuda:0')
COMMISSON: tensor(0.0778, device='cuda:0')
Test Accuracy: tensor(0.8989, device='cuda:0')



Test Mean IOU: tensor(0.2738, device='cuda:0')
Total IOU: tensor(0.4474, device='cuda:0')
OMISSON: tensor(0.2684, device='cuda:0')
COMMISSON: tensor(0.0778, device='cuda:0')
Test Accuracy: tensor(0.8977, device='cuda:0')



Test Mean IOU: tensor(0.1946, device='cuda:0')
Total IOU: tensor(0.1447, device='cuda:0')
OMISSON: tensor(0.0534, device='cuda:0')
COMMISSON: tensor(0.1209, device='cuda:0')
Test Accuracy: tensor(0.8611, device='cuda:0')
